In [13]:
# Importation des bibliothèque
import numpy as np 
import pandas as pd 
import datetime as dt

In [14]:
# Fais apparaitre l'ensemble des informations
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_column", 70)
pd.set_option("display.max_colwidth", 100)

In [15]:
# Lecture des données
data = pd.read_csv('../data/raw/dirty_cafe_sales.csv')
data = data.sort_values(by='Transaction Date')
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
7152,TXN_6566716,Coffee,ERROR,2.0,2.0,Credit Card,NaN,2023-01-01
8885,TXN_1581562,Coffee,2,2.0,4.0,Cash,In-store,2023-01-01
1806,TXN_2192787,Sandwich,5,4.0,20.0,Cash,In-store,2023-01-01
2244,TXN_5358805,Coffee,5,2.0,10.0,Digital Wallet,ERROR,2023-01-01
7285,TXN_1604072,Coffee,2,2.0,4.0,NaN,NaN,2023-01-01
...,...,...,...,...,...,...,...,...
9769,TXN_9686177,Cake,3,3.0,9.0,NaN,In-store,NaN
9833,TXN_5536245,Smoothie,4,4.0,16.0,Cash,NaN,NaN
9885,TXN_4659954,NaN,3,4.0,12.0,Credit Card,In-store,NaN
9931,TXN_8344810,Smoothie,2,4.0,8.0,NaN,UNKNOWN,NaN


In [16]:
#Diagnostic des valeur manquantes
missing = data.isnull().sum()
missing_pct = (data.isnull().sum() / len(data) * 100).round(2)

diagnostic = pd.DataFrame({
    'Valeurs manquantes': missing,
    'Pourcentage': missing_pct
})

print(diagnostic)
print(f"\nTotal de valeurs manquantes : {data.isnull().sum().sum()}")

                  Valeurs manquantes  Pourcentage
Transaction ID                     0         0.00
Item                             333         3.33
Quantity                         138         1.38
Price Per Unit                   179         1.79
Total Spent                      173         1.73
Payment Method                  2579        25.79
Location                        3265        32.65
Transaction Date                 159         1.59

Total de valeurs manquantes : 6826


In [17]:
#Vérification des doublons
duplicates = data.duplicated().sum()
print(f"Nombre de doublons : {duplicates}")

if duplicates > 0:
    print("Doublons détectés")
    data = data.drop_duplicates()
    print(f"Doublons supprimés. Nouvelles lignes : {len(data)}")
else:
    print("Aucun doublon détecté")

Nombre de doublons : 0
Aucun doublon détecté


In [18]:
# Remplacer les ERROR et UNKNOW par des valeur null afin de faciliter le traitement des données
data = data.replace(['ERROR', 'UNKNOWN'], np.nan)
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
7152,TXN_6566716,Coffee,NaN,2.0,2.0,Credit Card,NaN,2023-01-01
8885,TXN_1581562,Coffee,2,2.0,4.0,Cash,In-store,2023-01-01
1806,TXN_2192787,Sandwich,5,4.0,20.0,Cash,In-store,2023-01-01
2244,TXN_5358805,Coffee,5,2.0,10.0,Digital Wallet,NaN,2023-01-01
7285,TXN_1604072,Coffee,2,2.0,4.0,NaN,NaN,2023-01-01
...,...,...,...,...,...,...,...,...
9769,TXN_9686177,Cake,3,3.0,9.0,NaN,In-store,NaN
9833,TXN_5536245,Smoothie,4,4.0,16.0,Cash,NaN,NaN
9885,TXN_4659954,NaN,3,4.0,12.0,Credit Card,In-store,NaN
9931,TXN_8344810,Smoothie,2,4.0,8.0,NaN,NaN,NaN


<h3>Commentaire :</h3> <br>
Ici nous gérons les données incohérent en les remplacent pas une valeur null. Plus tard, ça sera plus facile pour nous de les traiter et les manipuler.

In [19]:
# Normalisation des types de données -> Ici string en float et date
data['Price Per Unit'] = data['Price Per Unit'].astype(float)
data['Total Spent'] = data['Total Spent'].astype(float)
data['Quantity'] = data['Quantity'].astype(float)
# format de la date en datetime
data['Transaction Date'] = pd.to_datetime(data['Transaction Date'])
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
7152,TXN_6566716,Coffee,NaN,2.0,2.0,Credit Card,NaN,2023-01-01
8885,TXN_1581562,Coffee,2.0,2.0,4.0,Cash,In-store,2023-01-01
1806,TXN_2192787,Sandwich,5.0,4.0,20.0,Cash,In-store,2023-01-01
2244,TXN_5358805,Coffee,5.0,2.0,10.0,Digital Wallet,NaN,2023-01-01
7285,TXN_1604072,Coffee,2.0,2.0,4.0,NaN,NaN,2023-01-01
...,...,...,...,...,...,...,...,...
9769,TXN_9686177,Cake,3.0,3.0,9.0,NaN,In-store,NaT
9833,TXN_5536245,Smoothie,4.0,4.0,16.0,Cash,NaN,NaT
9885,TXN_4659954,NaN,3.0,4.0,12.0,Credit Card,In-store,NaT
9931,TXN_8344810,Smoothie,2.0,4.0,8.0,NaN,NaN,NaT


<h3>Commentaire:</h3> <br> 
Ici nous changeons le types de colonne pour les colonnes -> Quantity, Price per unit, Total Spent et Transaction Date.
Toutes les colonnes sont en chaîne de caractères alors qu'ils sont censés être en colone numérique ou en datetime pour les dates.

In [20]:
# Extraire le datetime en plusieurs colonnes
data['Year'] = data['Transaction Date'].dt.year
data['Month'] = data['Transaction Date'].dt.month
data['Day'] = data['Transaction Date'].dt.day


# Afficher les nouvelles colonnes
print(f"\nNombre total de colonnes : {len(data.columns)}")
print(f"Nouvelles colonnes : {data.columns.tolist()}")


Nombre total de colonnes : 11
Nouvelles colonnes : ['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent', 'Payment Method', 'Location', 'Transaction Date', 'Year', 'Month', 'Day']


<h3>Commentaire :</h3> <br>
Il est important de créer des colonnes séparer pour la date afin de pouvoir manipuler la donnée plus facilement quand on voudra chercher sur une donnée sur la durée.

In [21]:
# Trouver le prix unitaire moyen par leur nom de 'Item' 
item_price_dict = data.set_index('Item')['Price Per Unit'].to_dict()
data['Price Per Unit'] = data.apply(lambda row: item_price_dict[row['Item']] if pd.isna(row['Price Per Unit']) else row['Price Per Unit'], axis=1)
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Year,Month,Day
7152,TXN_6566716,Coffee,NaN,2.0,2.0,Credit Card,NaN,2023-01-01,2023.0,1.0,1.0
8885,TXN_1581562,Coffee,2.0,2.0,4.0,Cash,In-store,2023-01-01,2023.0,1.0,1.0
1806,TXN_2192787,Sandwich,5.0,4.0,20.0,Cash,In-store,2023-01-01,2023.0,1.0,1.0
2244,TXN_5358805,Coffee,5.0,2.0,10.0,Digital Wallet,NaN,2023-01-01,2023.0,1.0,1.0
7285,TXN_1604072,Coffee,2.0,2.0,4.0,NaN,NaN,2023-01-01,2023.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
9769,TXN_9686177,Cake,3.0,3.0,9.0,NaN,In-store,NaT,NaN,NaN,NaN
9833,TXN_5536245,Smoothie,4.0,4.0,16.0,Cash,NaN,NaT,NaN,NaN,NaN
9885,TXN_4659954,NaN,3.0,4.0,12.0,Credit Card,In-store,NaT,NaN,NaN,NaN
9931,TXN_8344810,Smoothie,2.0,4.0,8.0,NaN,NaN,NaT,NaN,NaN,NaN


<h3>Commentaire:</h3> <br> 
Certains 'Item' étaient vide j'ai donc essayé de les remplir approximativement avec leur prix c'est à dire que les prix ont un seul prix dans le dataset et on va chercher le nom de l'item grâce au prix unitaire.

In [22]:
# Remplacer les valeur null par la quantité en faisant un Total spent / Price Per Unit pour trouver la quantité
data['Quantity'] = data['Total Spent'] / data['Price Per Unit']
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Year,Month,Day
7152,TXN_6566716,Coffee,1.0,2.0,2.0,Credit Card,NaN,2023-01-01,2023.0,1.0,1.0
8885,TXN_1581562,Coffee,2.0,2.0,4.0,Cash,In-store,2023-01-01,2023.0,1.0,1.0
1806,TXN_2192787,Sandwich,5.0,4.0,20.0,Cash,In-store,2023-01-01,2023.0,1.0,1.0
2244,TXN_5358805,Coffee,5.0,2.0,10.0,Digital Wallet,NaN,2023-01-01,2023.0,1.0,1.0
7285,TXN_1604072,Coffee,2.0,2.0,4.0,NaN,NaN,2023-01-01,2023.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
9769,TXN_9686177,Cake,3.0,3.0,9.0,NaN,In-store,NaT,NaN,NaN,NaN
9833,TXN_5536245,Smoothie,4.0,4.0,16.0,Cash,NaN,NaT,NaN,NaN,NaN
9885,TXN_4659954,NaN,3.0,4.0,12.0,Credit Card,In-store,NaT,NaN,NaN,NaN
9931,TXN_8344810,Smoothie,2.0,4.0,8.0,NaN,NaN,NaT,NaN,NaN,NaN


<h3>Commentaire:</h3> <br>
On s'occupe à présent des valeur nulles pour la colonne Quantity. Pour les trouver, on prend le total dépense divisé au prix unitaire (si les données sont disponible)

In [23]:
# Dernier Checkup des données
data.describe()

,Quantity,Price Per Unit,Total Spent,Transaction Date,Year,Month,Day
count,9498.000000,10000.000000,9498.000000,9540,9540.0,9540.000000,9540.000000
mean,3.019267,2.953350,8.924352,2023-07-01 23:00:31.698113024,2023.0,6.524004,15.738679
min,0.500000,1.000000,1.000000,2023-01-01 00:00:00,2023.0,1.000000,1.000000
25%,2.000000,2.000000,4.000000,2023-04-01 00:00:00,2023.0,4.000000,8.000000
50%,3.000000,3.000000,8.000000,2023-07-02 00:00:00,2023.0,7.000000,16.000000
75%,4.000000,4.000000,12.000000,2023-10-02 00:00:00,2023.0,10.000000,23.000000
max,6.250000,5.000000,25.000000,2023-12-31 00:00:00,2023.0,12.000000,31.000000
std,1.421518,1.279517,6.009919,NaN,0.0,3.449446,8.761003


In [24]:
# Exportation des données nettoyés ! :)
data.to_csv('../data/processed/cleaned_cafe_sales.csv', index=False)